In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as BS
import requests as r
import re
import time as t
from sqlalchemy import create_engine
import pymysql
!pip install ipynb-py-convert

In [ ]:
database_username = 'root'
database_password = '**********'
database_ip       = 'localhost'
database_name     = 'Linkedin_Jobs'

database_connection = create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.format(database_username, database_password,
                                                                             database_ip, database_name))

In [44]:
def scrape_linkedin_job_search(keywords,data=None):
    if data is None:   #Create it
        columns = ['Title', 'Company', 'Location','Link', 'Job Level','Job Type','Function', 'Sector','Description','Short Description']
        data = pd.DataFrame(columns=columns)
        
    data_all=data.copy()

    for i in range(4):
        titles = []
        companies = []
        locations = []
        links = []
        linkss=[]
        linksss=[]
        job_level=[]
        job_type=[]
        function=[]
        sector=[]
        description=[]
        
        url=f'https://www.linkedin.com/jobs/search/?keywords={keywords}&location=France&start={i*25}'
        page = r.get(url)
        soup = BS(page.content)

        for i in soup.select("div.result-card__contents"):
            title = i.findChild("h3", recursive=False)
            company = i.findChild("h4", recursive=False)
            location = i.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)

        links=soup.find_all(href=True)
        for i in links:
            if '/jobs/view' in str(i):
                linkss.append(i)
                
        for i in linkss:
            linksss.append(''.join(re.findall('href="(.+)"><span',str(i))))

        for i in range(len(linksss)):
            url=linksss[i]
            if i%5==0:
                t.sleep(1)
            page=r.get(linksss[i]).content
            soup=BS(page)
            job_level.append(' '.join([j.text for j in soup.select('.description > ul > li:nth-child(1) > span')]))
            job_type.append(' '.join([j.text for j in soup.select('.description > ul > li:nth-child(2) > span')]))
            function.append(' '.join([j.text for j in soup.select('.description > ul > li:nth-child(3) > span')]))
            sector.append(' '.join([j.text for j in soup.select('.description > ul > li:nth-child(4) > span')]))
            description.append(' '.join([j.text for j in soup.select('section.description > div')]))


        zipped = zip(titles, companies, locations, linksss, job_level, job_type, function, sector, description)
        for i in list(zipped):
            data_all=data_all.append({'Title' : i[0] , 'Company' : i[1], 'Location': i[2],'Link':i[3], 'Job Level': i[4], 'Job Type': i[5], 'Function': i[6], 'Sector': i[7], 'Description':i[8]} , ignore_index=True)
        
    data_all=data_all.drop_duplicates(keep='first',subset=['Company','Description']).reset_index(drop=True)
    data_new=data_all.copy()
    if data_all.shape[0]>data.shape[0]:
        rows_to_drop=[i for i in range(data.shape[0])]
        data_new=data_new.drop(rows_to_drop).reset_index(drop=True)
    data_new['Description']=data_new['Description'].apply(lambda x:x.lower())
    
    data_new['Short Description']=''

    for i in range(data_new.shape[0]):
        if 'sql' in data_new['Description'][i]:
            data_new['Short Description'][i]='SQL, '
        if 'python' in data_new['Description'][i]:
            data_new['Short Description'][i]=str(data_new['Short Description'][i])+'Python, '
        if 'tableau' in data_new['Description'][i]:
            data_new['Short Description'][i]=str(data_new['Short Description'][i])+'Tableau, '
        if 'scikit' in data_new['Description'][i] or 'sklearn' in data_new['Description'][i]:
            data_new['Short Description'][i]=str(data_new['Short Description'][i])+'SkLearn, '
        if 'git' in data_new['Description'][i] :
            data_new['Short Description'][i]=str(data_new['Short Description'][i])+'git, '
        if 'pandas' in data_new['Description'][i] :
            data_new['Short Description'][i]=str(data_new['Short Description'][i])+'Pandas, '
        if 'numpy' in data_new['Description'][i] :
            data_new['Short Description'][i]=str(data_new['Short Description'][i])+'Numpy, '
        if 'seaborn' in data_new['Description'][i] or 'matplolib' in data_new['Description'][i] or 'visualization' in data_new['Description'][i] or 'visualisation' in data_new['Description'][i]:
            data_new['Short Description'][i]=str(data_new['Short Description'][i])+'Data Viz, ' 
    data.to_sql('table_name_for_df',database_connection, if_exists='replace', index=False)
    !ipython nbconvert --to=python Linkedin_Scraping.ipynb
    
#     data_all.to_sql('All_Offers',if_exists='replace', index = False)
#     data_new.to_sql('New_Offers',if_exists='replace', index = False)
    return data_new

In [45]:
data1=scrape_linkedin_job_search('data&20analyse')

In [46]:
data1.shape

(25, 10)

In [47]:
data2=scrape_linkedin_job_search('data&20analyse',data1)

In [48]:
data2.shape

(65, 10)

In [49]:
data1.append(data2).drop_duplicates(subset=['Description','Company'],keep='first')

,Title,Company,Location,Link,Job Level,Job Type,Function,Sector,Description,Short Description
0,Data Scientist (H/F),Groupe Karavel - Promovacances,"Paris, FR",https://fr.linkedin.com/jobs/view/data-scienti...,Premier emploi,Temps plein,Ingénierie Technologies de l’information,"Compagnie aérienne/Aviation Loisirs, voyages e...",n°1 de la vente de séjours en ligne avec plus ...,
1,Data Engineer,OECD - OCDE,"Paris Area, France",https://fr.linkedin.com/jobs/view/data-enginee...,Manager,Temps plein,Études/recherche Rédaction/édition Management ...,Affaires étrangères Collectivités publiques et...,the public affairs and communications director...,"SQL, Tableau, git,"
2,data analyst H/F,La Banque Postale,"Limoges, Nouvelle-Aquitaine, France",https://fr.linkedin.com/jobs/view/data-analyst...,Confirmé,Temps plein,Technologies de l’information Finance Analyste,Banques,"vous maîtrisez des outils data mining (sas, i...",
3,Assistant Data Analyst,Total,"Pau, FR",https://fr.linkedin.com/jobs/view/assistant-da...,Confirmé,Temps plein,Technologies de l’information,Technologies et services de l’information Recr...,description du postele groupe total est un act...,"SQL, Python,"
4,Data Analyst,IQVIA,"Angers Area, France",https://fr.linkedin.com/jobs/view/data-analyst...,Manager,Temps plein,Analyste Sciences Études/recherche,Hôpitaux et centres de soins Biotechnologie In...,"data analyst, real world & analytics solutions...","SQL, Python, Tableau, Data Viz,"
5,DATA ANALYST,Catalina Marketing France,"Boulogne-Billancourt, Île-de-France, France",https://fr.linkedin.com/jobs/view/data-analyst...,Stagiaire,Temps plein,Technologies de l’information,Marketing et publicité,visitez notre page 😊 https://www.welcometothej...,"SQL, Python,"
6,ENGINEER DATA SCIENTIST,ALTEN,"Toulouse, Occitanie, France",https://fr.linkedin.com/jobs/view/engineer-dat...,Confirmé,Temps plein,Ingénierie,Aéronautique et aérospatiale,responsibilities : · analyze raw data: asse...,"SQL, Python, Tableau, Data Viz,"
7,Data Analyst,Solocal,Paris et périphérie,https://fr.linkedin.com/jobs/view/data-analyst...,Confirmé,Temps plein,Technologies de l’information,Marketing et publicité,contextela/le data analyst travaillera en étro...,"SQL, Python, Tableau, git, Data Viz,"
8,DATA ANALYST,Sogia Systeme,"Paris, FR",https://fr.linkedin.com/jobs/view/data-analyst...,Premier emploi,Temps plein,Technologies de l’information,Technologies et services de l’information Logi...,intitulé du poste : data analystdescription du...,"SQL, Python, Data Viz,"
9,Data Analyst,Le Point,"Paris, Île-de-France, France",https://fr.linkedin.com/jobs/view/data-analyst...,Non pertinent,Temps plein,Analyste,Presse écrite,le point recherche un(e) data analystrattaché(...,"SQL, Python, Tableau, Data Viz,"


In [23]:
data1.Link[7]

'https://fr.linkedin.com/jobs/view/data-analyst-at-solocal-1667326695?refId=9f97a5ef-bbc0-4d85-9dc7-3d45f65e86fd&amp;position=8&amp;pageNum=0&amp;trk=guest_job_search_job-result-card_result-card_full-click'

In [24]:
data1.Link[82]

'https://fr.linkedin.com/jobs/view/data-analyst-at-solocal-1667326695?refId=91f13471-6a5b-4cb7-aa00-92a6e80a9d1a&amp;position=8&amp;pageNum=0&amp;trk=guest_job_search_job-result-card_result-card_full-click'

In [25]:
data2.Link[28]

'https://fr.linkedin.com/jobs/view/data-analyst-at-solocal-1667326695?refId=c6d24d1c-471a-440f-b868-b76aee11348e&amp;position=4&amp;pageNum=0&amp;trk=guest_job_search_job-result-card_result-card_full-click'

In [26]:
data2.Link[3]

'https://fr.linkedin.com/jobs/view/data-analyst-at-solocal-1667326695?refId=5602330c-4304-4cdf-83db-dd6f2c5da921&amp;position=4&amp;pageNum=0&amp;trk=guest_job_search_job-result-card_result-card_full-click'